# Example 6.15

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display, Math
from scipy import stats
from scipy import linalg as la

In [2]:
def load_data() -> pd.DataFrame:
    df1 = pd.read_excel(r'..\..\data\Table6.5.xlsx')
    df1['Group'] = 1

    df2 = pd.read_excel(r'..\..\data\Table6.6.xlsx')
    df2['Group'] = 2

    return pd.concat([df1, df2])

In [3]:
df = load_data()

In [4]:
data_cols = ['Initial','1 year','2 year','3 year']
X1 = df.loc[df.Group.eq(1), data_cols].to_numpy()
X2 = df.loc[df.Group.eq(2), data_cols].to_numpy()
X = np.vstack([X1,X2])

In [5]:
n1 = X1.shape[0]
n2 = X2.shape[0]
N = n1 + n2
g = 2
p = len(data_cols)

In [6]:
q = 2  # Parabola.
parab = lambda t: np.array([1, t, t**2])
B = np.array([parab(i) for i in range(4)])
B

array([[1, 0, 0],
       [1, 1, 1],
       [1, 2, 4],
       [1, 3, 9]])

$$
    \bar{\textbf{X}}
    =
    \left[
        \begin{array}{cc}
            \bar{\textbf{x}}_{1} & \bar{\textbf{x}}_{2}
        \end{array}
    \right]
    =
    \left[
        \begin{array}{cc}
            \bar{\textbf{x}}_{11} & \bar{\textbf{x}}_{21} \\
            \bar{\textbf{x}}_{12} & \bar{\textbf{x}}_{22} \\
            \bar{\textbf{x}}_{13} & \bar{\textbf{x}}_{23} \\
            \bar{\textbf{x}}_{14} & \bar{\textbf{x}}_{24}
        \end{array}
    \right]
$$

In [7]:
xbar1 = np.mean(X1,axis=0)[:,np.newaxis]
xbar2 = np.mean(X2,axis=0)[:,np.newaxis]

$$
\textbf{S}_{\text{pooled}}
\frac{1}{(N - g)}
((n_{1} - 1)\textbf{S}_{1} + \cdots + (n_{g} - 1)\textbf{S}_{g})
=
\frac{1}{(N - g)}
\textbf{W}
$$

In [8]:
S_pooled = (1/(N-g))*((n1- 1)*np.cov(X1,rowvar=False) + (n2- 1)*np.cov(X2,rowvar=False))
S_pooled

array([[95.25108621, 91.74997414, 81.70028448, 80.54869828],
       [91.74997414, 95.03478305, 80.81083477, 80.27450144],
       [81.70028448, 80.81083477, 79.36943822, 72.36358764],
       [80.54869828, 80.27450144, 72.36358764, 78.53281753]])

$$
    {\left(
        \textbf{B}^{\prime}
        \textbf{S}_{\text{pooled}}^{-1}
        \textbf{B}
    \right)}^{-1}
$$

In [9]:
la.inv(B.T @ la.inv(S_pooled) @ B)

array([[93.17444074, -5.83684974,  0.21835566],
       [-5.83684974,  9.56994602, -3.02401088],
       [ 0.21835566, -3.02401088,  1.10507904]])

$$
\widehat{\text{Cov}}(\hat{\bm{\beta}}_{\ell})
=
\frac{k}{n_{\ell}}
{\left(
    \textbf{B}^{\prime}
    \textbf{S}_{\text{pooled}}^{-1}
    \textbf{B}
\right)}^{-1}
\hspace{0.4cm}
\text{for}
\hspace{0.4cm}
\ell = 1, 2, \dots, g
$$

$$
k
=
\frac{(N-g)(N-g-1)}{(N-g-p+q)(N-g-p+q+1)}
$$

In [10]:
k = ((N - g)*(N - g - 1))/((N - g - p + q)*(N - g - p + q + 1))

Group 1

In [11]:
cov_est_group_1 = (k/n1)*la.inv(B.T @ la.inv(S_pooled) @ B)
cov_est_group_1

array([[ 6.67175008, -0.41794727,  0.01563534],
       [-0.41794727,  0.68525539, -0.21653411],
       [ 0.01563534, -0.21653411,  0.07912912]])

Group 2

In [12]:
cov_est_group_2 = (k/n2)*la.inv(B.T @ la.inv(S_pooled) @ B)
cov_est_group_2

array([[ 6.2547657 , -0.39182556,  0.01465813],
       [-0.39182556,  0.64242693, -0.20300073],
       [ 0.01465813, -0.20300073,  0.07418355]])

$$
\hat{\bm{\beta}}_{\ell}
=
{\left(
    \textbf{B}^{\prime}
    \textbf{S}_{\text{pooled}}^{-1}
    \textbf{B}
\right)}^{-1}
\textbf{B}^{\prime}
\textbf{S}_{\text{pooled}}^{-1}
\bar{\textbf{X}}_{\ell}
\hspace{0.4cm}
\text{for}
\hspace{0.4cm}
\ell = 1, 2, \dots, g
$$

In [13]:
beta_est = np.hstack([la.inv(B.T @ la.inv(S_pooled) @ B) @ B.T @ la.inv(S_pooled) @ xbar1,
                      la.inv(B.T @ la.inv(S_pooled) @ B) @ B.T @ la.inv(S_pooled) @ xbar2])

In [14]:
# The standard deviations are in parenthesis.
display(Math(r'\text{The estimated growth curves are}'))
display(Math(r'\begin{array}{lccccc}'
             fr'\text{{Control group:}}   & {beta_est[0,0]:.2f} & + & {beta_est[1,0]:.2f}t & - & {np.abs(beta_est[2,0]):.2f}t^{2} \\'
             fr' & ({np.sqrt(cov_est_group_1[0,0]):.2f}) & & ({np.sqrt(cov_est_group_1[1,1]):.2f}) & & ({np.sqrt(cov_est_group_1[2,2]):.2f}) \\'
             fr'\text{{Treatment group:}} & {beta_est[0,1]:.2f} & + & {beta_est[1,1]:.2f}t & - & {np.abs(beta_est[2,1]):.2f}t^{2} \\'
             fr' & ({np.sqrt(cov_est_group_2[0,0]):.2f}) & & ({np.sqrt(cov_est_group_2[1,1]):.2f}) & & ({np.sqrt(cov_est_group_2[2,2]):.2f})'
             r'\end{array}'
             ))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
def compute_Wq(X: np.ndarray, B: np.ndarray, beta_est: np.array):
    n, p = X.shape
    results = np.zeros((p, p))
    for i in range(n):
        obs = X[i,:].copy()[:,np.newaxis]
        results += (obs - B @ beta_est) @ (obs - B @ beta_est).T
    return results

$$
    \textbf{W}_{q}
    =
    \sum_{\ell = 1}^{g}
    \sum_{j = 1}^{n_{\ell}}
    (\textbf{X}_{\ell j} - \textbf{B} \hat{\bm{\beta}}_{\ell})
    {(\textbf{X}_{\ell j} - \textbf{B} \hat{\bm{\beta}}_{\ell})}^{\prime}
$$

In [16]:
# q is the degree of the polynomial. Here's that's 2.
W_2 = compute_Wq(X1, B, beta_est[:,[0]]) + compute_Wq(X2, B, beta_est[:,[1]])
W_2

array([[2781.01741477, 2698.58861289, 2363.22837387, 2362.2535235 ],
       [2698.58861289, 2832.42971057, 2331.23519008, 2381.15981267],
       [2363.22837387, 2331.23519008, 2303.68665144, 2089.99619748],
       [2362.2535235 , 2381.15981267, 2089.99619748, 2314.48553872]])

In [17]:
def compute_W(X: np.ndarray):
    n, p = X.shape
    xbar = np.mean(X,axis=0)[:,np.newaxis]
    results = np.zeros((p, p))
    for i in range(n):
        obs = X[i,:].copy()[:,np.newaxis]
        results += (obs - xbar) @ (obs - xbar).T
    return results

$$
    \textbf{W}
    =
    \sum_{\ell}^{g}
    \sum_{j}^{n_{\ell}}
    (\textbf{x}_{\ell j} - \bar{\textbf{x}}_{\ell})
    {(\textbf{x}_{\ell j} - \bar{\textbf{x}}_{\ell})}^{\prime}
$$

In [18]:
W = compute_W(X1) + compute_W(X2)
W

array([[2762.2815    , 2660.74925   , 2369.30825   , 2335.91225   ],
       [2660.74925   , 2756.00870833, 2343.51420833, 2327.96054167],
       [2369.30825   , 2343.51420833, 2301.71370833, 2098.54404167],
       [2335.91225   , 2327.96054167, 2098.54404167, 2277.45170833]])

In [19]:
lmbda_star = la.det(W) / la.det(W_2)
test_value = -(N - 0.5*(p - q + g))*np.log(lmbda_star)
display(Math(r'-\left( N - \frac{1}{2} (p - q + g) \right) \ln \Lambda^{\star}'
             '='
             fr'-\left( {N} - \frac{{1}}{{2}} ({p} - {q} + {g}) \right) \ln {lmbda_star:.4f}'
             '='
             f'{test_value:.2f}'
             ))

<IPython.core.display.Math object>

In [20]:
alpha = 0.01
chi2_crit = stats.chi2.ppf(1-alpha, df=(p-(g+1))*2)
display(Math(fr'\chi_{{ ({p} - ({g}+1))2 }}^{{2}}({alpha})'
             '='
             f'{chi2_crit:.2f}'
             ))

<IPython.core.display.Math object>

In [21]:
if test_value > chi2_crit:
    display(Math(fr'\text{{We have that }} X^{{2\star}} = {test_value:.2f} > \chi_{{ {(p-(g+1))*2} }}^{2}({alpha}) = '
                 fr'{chi2_crit:.2f} \text{{, so we would reject the null hypothesis that }} '
                 r'\text{the polynomial is adequate.}'
                 ))
else:
    display(Math(fr'\text{{We have that }} X^{{2\star}} = {test_value:.2f} < \chi_{{ {(p - (g+1))*2} }}^{2}({alpha}) = '
                 fr'{chi2_crit:.2f} \text{{, so we would fail to reject the null hypothesis that }} '
                 r'\text{the polynomial is adequate.}'
                ))

<IPython.core.display.Math object>

In [22]:
# The survival finction is 1 - CDF, so this gives the same answer: 1 - stats.chi2.cdf(test_value, df=(p-(g+1))*2)
p_value = stats.chi2.sf(test_value, df=(p-(g+1))*2)
display(Math(fr'\text{{ The p-value for the test is }} {p_value:.4f}.'))

<IPython.core.display.Math object>

# From the book Advanced Linear Models by Ronald Christensen

These estimates from the book are broken up so we compute the estimates for each group separately, but we can also compute the estimates at once. For this we'd use the generalized least squares estimate for $\bm{\beta}$ (all of our parameters), where the formula the formula can be found in Theorem 2.7.1 (b) on page 40 of the book Plane Answers to Complex Questions 5th edition by Ronald Christensen. Along with our model setup in 11.1.2 on page 424 of the book Advanced Linear Models 3rd edition, also by Ronald Christensen to get...
$$
\text{Vec}(\hat{\bm{\Gamma}})
=
{\left(
    {[ \textbf{Z} \otimes \textbf{X} ]}^{\prime}
    {( \bm{\Sigma}^{-1} \otimes \textbf{I}_{n} )}
    [ \textbf{Z} \otimes \textbf{X} ]
\right)}^{-1}
{[ \textbf{Z} \otimes \textbf{X} ]}^{\prime}
{( \bm{\Sigma}^{-1} \otimes \textbf{I}_{n} )}
\text{Vec}(\textbf{Y})
$$
The notation here differs from the book we're using though. We need to replace $\textbf{Z}$ above with $\textbf{B}$ and replace $\textbf{Y}$ above with $\textbf{X}$. The matrix $\textbf{X}$ above is the design matix, which identify which group an observation belongs to. We create that with
$$
\textbf{E}
=
{\left[
    \begin{array}{cc}
        \textbf{1}_{n_{1}} \textbf{0}_{n_{1}} \\
        \textbf{0}_{n_{2}} \textbf{1}_{n_{2}}
    \end{array}
\right]}_{N \times 2}
$$
This consists of $N = n_{1} + n_{2}$ rows and two columns. The first column has $n_{1}$ values of 1 (for the control group observations) followed by $n_{2}$ values of 0. The second column has $n_{1}$ values of 0 followed by $n_{2}$ values of 1 (for the treatment group observations). The last thing is that we don't have the population covariance matrix $\bm{\Sigma}$, but estimate it with $\textbf{S}_{\text{pooled}}$, so we can replace $\bm{\Sigma}$ with $\textbf{S}_{\text{pooled}}$. Using $\textbf{S}_{\text{pooled}}$ is a consistent estimator, so for a big $n$ we have and estimate close to the population. The estimates will be created below by computing...
$$
\text{Vec}(\hat{\bm{\Gamma}})
=
{\left(
    {[ \textbf{B} \otimes \textbf{E} ]}^{\prime}
    {( \textbf{S}_{\text{pooled}}^{-1} \otimes \textbf{I}_{N} )}
    [ \textbf{B} \otimes \textbf{E} ]
\right)}^{-1}
{[ \textbf{B} \otimes \textbf{E} ]}^{\prime}
{( \textbf{S}_{\text{pooled}}^{-1} \otimes \textbf{I}_{N} )}
\text{Vec}(\textbf{X})
$$
The estimates, $\text{Vec}(\hat{\bm{\Gamma}})$, will come out stacked on one another. First two are intercepts, next two are slope, and last two are quadratic parameters.


In [23]:
# Create design matrix X. Column 1 is group 1. Column 2 is group 2.
E = np.vstack([np.hstack([np.ones((n1,1)),  np.zeros((n1,1))]),
               np.hstack([np.zeros((n2,1)), np.ones((n2,1))])])

In [24]:
la.inv(np.kron(B,E).T @ la.inv(np.kron(S_pooled, np.eye(N))) @ np.kron(B,E)) @ np.kron(B,E).T @ la.inv(np.kron(S_pooled, np.eye(N))) @ X.T.reshape([-1,1])

array([[73.07012323],
       [70.13867065],
       [ 3.64435751],
       [ 4.09004971],
       [-2.02736314],
       [-1.85343205]])